# **PREDICTING CREDIT CARD APPROVALS**

![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


## 1. Explorative Data Analysis - Missing Values, Data-Types, 

In [2]:
cc_apps.iloc[:,13].unique()

array(['+', '-'], dtype=object)

In [3]:
cc_apps.columns

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], dtype='int64')

In [4]:
for col in cc_apps.columns:
    print(col,": ",cc_apps.iloc[:,col].unique(), '\n')

0 :  ['b' 'a' '?'] 

1 :  ['30.83' '58.67' '24.50' '27.83' '20.17' '32.08' '33.17' '22.92' '54.42'
 '42.50' '22.08' '29.92' '38.25' '48.08' '45.83' '36.67' '28.25' '23.25'
 '21.83' '19.17' '25.00' '47.75' '27.42' '41.17' '15.83' '47.00' '56.58'
 '57.42' '42.08' '29.25' '42.00' '49.50' '36.75' '22.58' '27.25' '23.00'
 '27.75' '54.58' '34.17' '28.92' '29.67' '39.58' '56.42' '54.33' '41.00'
 '31.92' '41.50' '23.92' '25.75' '26.00' '37.42' '34.92' '34.25' '23.33'
 '23.17' '44.33' '35.17' '43.25' '56.75' '31.67' '23.42' '20.42' '26.67'
 '36.00' '25.50' '19.42' '32.33' '34.83' '38.58' '44.25' '44.83' '20.67'
 '34.08' '21.67' '21.50' '49.58' '27.67' '39.83' '?' '37.17' '25.67'
 '34.00' '49.00' '62.50' '31.42' '52.33' '28.75' '28.58' '22.50' '28.50'
 '37.50' '35.25' '18.67' '54.83' '40.92' '19.75' '29.17' '24.58' '33.75'
 '25.42' '37.75' '52.50' '57.83' '20.75' '39.92' '24.75' '44.17' '23.50'
 '47.67' '22.75' '34.42' '28.42' '67.75' '47.42' '36.25' '32.67' '48.58'
 '33.58' '18.83' '26.92' '31.

In [5]:
cc_apps_nans_replaced = cc_apps.replace("?", np.NaN)

In [6]:
cc_apps_nans_replaced.isna().sum()

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13     0
dtype: int64

In [7]:
len(cc_apps)

690

In [8]:
cc_apps.dtypes

0      object
1      object
2     float64
3      object
4      object
5      object
6      object
7     float64
8      object
9      object
10      int64
11     object
12      int64
13     object
dtype: object

In [9]:
cc_apps_nans_replaced[1] = cc_apps_nans_replaced[1].astype(float)

In [10]:
cc_apps_nans_replaced.dtypes

0      object
1     float64
2     float64
3      object
4      object
5      object
6      object
7     float64
8      object
9      object
10      int64
11     object
12      int64
13     object
dtype: object

In [11]:
for col in cc_apps_nans_replaced.columns:
    # Check if the column is of object type
    if cc_apps_nans_replaced[col].dtypes == "object":
        # Impute with the most frequent value
        cc_apps_nans_replaced[col] = cc_apps_nans_replaced[col].fillna(
            cc_apps_nans_replaced[col].value_counts().index[0]
        )
    else:
        cc_apps_nans_replaced[col] = cc_apps_nans_replaced[col].fillna(cc_apps_nans_replaced[col].mean())


In [12]:
cc_apps_nans_replaced.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64

In [13]:
cc_apps_nans_replaced.describe()

,1,2,7,10,12
count,690.000000,690.000000,690.000000,690.00000,690.000000
mean,31.568171,4.758725,2.223406,2.40000,1017.385507
std,11.853273,4.978163,3.346513,4.86294,5210.102598
min,13.750000,0.000000,0.000000,0.00000,0.000000
25%,22.670000,1.000000,0.165000,0.00000,0.000000
50%,28.625000,2.750000,1.000000,0.00000,5.000000
75%,37.707500,7.207500,2.625000,3.00000,395.500000
max,80.250000,28.000000,28.500000,67.00000,100000.000000


In [14]:
cc_apps_nans_replaced.head(30)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.250,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.040,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.500,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.750,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.710,t,f,0,s,0,+
5,b,32.08,4.000,u,g,m,v,2.500,t,f,0,g,0,+
6,b,33.17,1.040,u,g,r,h,6.500,t,f,0,g,31285,+
7,a,22.92,11.585,u,g,cc,v,0.040,t,f,0,g,1349,+
8,b,54.42,0.500,y,p,k,h,3.960,t,f,0,g,314,+
9,b,42.50,4.915,y,p,w,v,3.165,t,f,0,g,1442,+


In [15]:
cc_preprocessed = pd.get_dummies(cc_apps_nans_replaced, drop_first = True, dtype = int)
cc_preprocessed.head(10)

,1,2,7,10,12,0_b,3_u,3_y,4_gg,4_p,...,6_j,6_n,6_o,6_v,6_z,8_t,9_t,11_p,11_s,13_-
0,30.83,0.000,1.250,1,0,1,1,0,0,0,...,0,0,0,1,0,1,1,0,0,0
1,58.67,4.460,3.040,6,560,0,1,0,0,0,...,0,0,0,0,0,1,1,0,0,0
2,24.50,0.500,1.500,0,824,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,27.83,1.540,3.750,5,3,1,1,0,0,0,...,0,0,0,1,0,1,1,0,0,0
4,20.17,5.625,1.710,0,0,1,1,0,0,0,...,0,0,0,1,0,1,0,0,1,0
5,32.08,4.000,2.500,0,0,1,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
6,33.17,1.040,6.500,0,31285,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
7,22.92,11.585,0.040,0,1349,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
8,54.42,0.500,3.960,0,314,1,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
9,42.50,4.915,3.165,0,1442,1,0,1,0,1,...,0,0,0,1,0,1,0,0,0,0


In [16]:
cc_preprocessed2= pd.get_dummies(cc_apps_nans_replaced, drop_first = False, dtype = int)
cc_preprocessed2.head(10)

,1,2,7,10,12,0_a,0_b,3_l,3_u,3_y,...,6_z,8_f,8_t,9_f,9_t,11_g,11_p,11_s,13_+,13_-
0,30.83,0.000,1.250,1,0,0,1,0,1,0,...,0,0,1,0,1,1,0,0,1,0
1,58.67,4.460,3.040,6,560,1,0,0,1,0,...,0,0,1,0,1,1,0,0,1,0
2,24.50,0.500,1.500,0,824,1,0,0,1,0,...,0,0,1,1,0,1,0,0,1,0
3,27.83,1.540,3.750,5,3,0,1,0,1,0,...,0,0,1,0,1,1,0,0,1,0
4,20.17,5.625,1.710,0,0,0,1,0,1,0,...,0,0,1,1,0,0,0,1,1,0
5,32.08,4.000,2.500,0,0,0,1,0,1,0,...,0,0,1,1,0,1,0,0,1,0
6,33.17,1.040,6.500,0,31285,0,1,0,1,0,...,0,0,1,1,0,1,0,0,1,0
7,22.92,11.585,0.040,0,1349,1,0,0,1,0,...,0,0,1,1,0,1,0,0,1,0
8,54.42,0.500,3.960,0,314,0,1,0,0,1,...,0,0,1,1,0,1,0,0,1,0
9,42.50,4.915,3.165,0,1442,0,1,0,0,1,...,0,0,1,1,0,1,0,0,1,0


In [17]:
print(cc_preprocessed.columns)
print(cc_preprocessed2.columns)

Index([     1,      2,      7,     10,     12,  '0_b',  '3_u',  '3_y', '4_gg',
        '4_p',  '5_c', '5_cc',  '5_d',  '5_e', '5_ff',  '5_i',  '5_j',  '5_k',
        '5_m',  '5_q',  '5_r',  '5_w',  '5_x', '6_dd', '6_ff',  '6_h',  '6_j',
        '6_n',  '6_o',  '6_v',  '6_z',  '8_t',  '9_t', '11_p', '11_s', '13_-'],
      dtype='object')
Index([     1,      2,      7,     10,     12,  '0_a',  '0_b',  '3_l',  '3_u',
        '3_y',  '4_g', '4_gg',  '4_p', '5_aa',  '5_c', '5_cc',  '5_d',  '5_e',
       '5_ff',  '5_i',  '5_j',  '5_k',  '5_m',  '5_q',  '5_r',  '5_w',  '5_x',
       '6_bb', '6_dd', '6_ff',  '6_h',  '6_j',  '6_n',  '6_o',  '6_v',  '6_z',
        '8_f',  '8_t',  '9_f',  '9_t', '11_g', '11_p', '11_s', '13_+', '13_-'],
      dtype='object')


## 2. Preprocessing

- 12th Row has large STD and could affect prediction of models such as KNN

In [18]:
# Type A: Drop_first df
features = cc_preprocessed.drop(columns = ['13_-']).values
target = cc_preprocessed['13_-'].values

# Type B: Full Dummies df
features2 = cc_preprocessed2.drop(columns = ['13_+', '13_-']).values
target2 = cc_preprocessed2[['13_+', '13_-']].values


print(features2[:3],'\n\n',features[:3])
print('\n\n', target2[:3], target[:3])

[[3.083e+01 0.000e+00 1.250e+00 1.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 1.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 0.000e+00 1.000e+00 0.000e+00 1.000e+00 1.000e+00 0.000e+00
  0.000e+00]
 [5.867e+01 4.460e+00 3.040e+00 6.000e+00 5.600e+02 1.000e+00 0.000e+00
  0.000e+00 1.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 1.000e+00 0.000e+00 1.000e+00 1.000e+00 0.000e+00
  0.000e+00]
 [2.450e+01 5.000e-01 1.500e+00 0.000e+00 8.240e+02 1.000e+00 0.000e+00
  0.000e+00 1.000e+00 0.000e+00 1.000e

In [19]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import confusion_matrix
# from sklearn.model_selection import GridSearchCV

In [20]:
# Pipeline
from sklearn.pipeline import Pipeline

scaler = StandardScaler()
logreg = LogisticRegression()


In [21]:
X_train,X_test, y_train, y_test = train_test_split(features, target, random_state = 14)

scaled_train = scaler.fit_transform(X_train)
scaled_test = scaler.transform(X_test)

logreg.fit(scaled_train, y_train)

LogisticRegression()

In [22]:
pred_test = logreg.predict(scaled_train)
confusion_matrix(y_train, pred_test)

# # Use logreg to predict instances from the training set
# y_train_pred = logreg.predict(rescaledX_train)

# # Print the confusion matrix of the logreg model
# print(confusion_matrix(y_train, y_train_pred))

array([[217,  18],
       [ 36, 246]])

## Finding Best Scoring Hyperparameters

In [23]:
# Define the grid of values for tol and max_iter
tol = [0.01, 0.001, 0.0001]
max_iter = [100, 150, 200]

param_grid = {'tol': tol, 'max_iter': max_iter}
grid = GridSearchCV(estimator = logreg, param_grid = param_grid, cv = 5 )

grid_fitting = grid.fit(scaled_train, y_train)
grid_best_score = grid_fitting.best_score_
grid_best_param = grid_fitting.best_params_
print('Best: Parameters: ',grid_best_param, '\nBest Score:', grid_best_score)

Best: Parameters:  {'max_iter': 100, 'tol': 0.01} 
Best Score: 0.8762322628827484


In [24]:
# Extract the best model and evaluate it on the test set
best_model = grid_fitting.best_estimator_
print(best_model,'\n')
best_score =  best_model.score(scaled_test, y_test)

print("Accuracy of logistic regression classifier: ", best_score)

LogisticRegression(tol=0.01) 

Accuracy of logistic regression classifier:  0.8323699421965318


___
___